# How to use nodejs express?

In [1]:
var exec = require('child_process').execSync;var fs = require('fs');var sourceCmd = exec('npm install jsdom express request mime', {stdio: [0, 1, 2]});var express = require('express');var mime = require('mime');var path = require('path');var request = require('request');var JSDOM = require('jsdom').JSDOM;var output = path.join(process.cwd(), 'output');var port = process.env.PORT || 3000;var host = 'localhost';var query = '.w-gallery-list img';$$.done('express is ready');

'express is ready'

How to start a proxy using express?

How to start a proxy on {port}?



In [2]:
if (typeof server != 'undefined') {    server.close();}var app = express();app.use('/', function (req, res) {    var url = host + req.url;    req.pipe(request(url)).pipe(res);});$$.async();try {    var server = app.listen(port, () => $$.done('server up and running on port ' + port));    server.on('error', (e) => $$.done(e));} catch (e) {    $$.done(e);}

{ Error: listen EADDRINUSE :::3000
    at Object.exports._errnoException (util.js:1026:11)
    at exports._exceptionWithHostPort (util.js:1049:20)
    at Server._listen2 (net.js:1253:14)
    at listen (net.js:1289:10)
    at Server.listen (net.js:1385:5)
    at EventEmitter.listen (/Users/briancullinan/jupytangular2/node_modules/express/lib/application.js:618:24)
    at evalmachine.<anonymous>:12:22
    at ContextifyScript.Script.runInThisContext (vm.js:25:33)
    at Object.exports.runInThisContext (vm.js:77:17)
    at run ([eval]:613:19)
  code: 'EADDRINUSE',
  errno: 'EADDRINUSE',
  syscall: 'listen',
  address: '::',
  port: 3000 }


How to display the output from express?

How to test express is working with HTML?

In [3]:
$$.mime({    'text/html': `<iframe id="sosmethod" name="sosmethod"     style="height:600px; width:100%; border:none;"     src="http://${host}:${port}/"></iframe>`});

How to run a DOM query on a remote HTML page?

In [4]:
$$.async();var images = [];new Promise((resolve, reject) => {    request('http://' + host + ':' + port, (err, res, body) => {        if (err) {            reject(err)        }        else {            resolve(body)        }    });}).then((body) => {    var nodeList = (new JSDOM(body)).window.document.querySelectorAll(query);    return Array.prototype.slice.call(nodeList, 0)        .map(i => i.getAttribute('src'));}).then(sources => {    images = sources;    $$.done(sources);}).catch(e => $$.done(e));

How to save a list of image elements?

How to display a list of image elements in markdown?

How to {output} image elements in the requested {format}?


In [16]:
var html = '';var htmlPrint = '';var requests = images.map((src) => {    var filename = src.split('/').pop();    return new Promise((resolve, reject) => {        request(src, {encoding: 'binary'}, (err, res, body) => {            if (err) {                reject(err)            }            else {                resolve(body)            }        });    }).then(body => {        return new Promise((resolve, reject) => {            fs.writeFile(                path.join(output, filename),                body,                'binary',                function (err) {                    if (err) reject(err);                    else resolve({filename: filename, data: body});                });        })    });});$$.async();Promise.all(requests).then(images => {    images.forEach((i) => {        var filename = i.filename.split('/').pop();        var ext = mime.lookup(i.filename);        html += '<img src="data:' + ext + ';base64,' + (new Buffer(i.data, 'binary')).toString('base64') + '" />';        htmlPrint += '<li><img src="/assets/' + filename + '" /></li>\n';    });    $$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOutput:\n' + html});}).catch(e => $$.done(e));

How to stop express server?

In [6]:
if (typeof server !== 'undefined') {    server.close();}